# Most Popular Car by State

In [1]:
# Import dependencies 
from bs4 import BeautifulSoup as bs
import requests
import os
import pandas as pd
import json
import sqlite3 as sq

### Scrape "USA Today" Site for State and Model Data

In [2]:
# URL of page to be scraped
url = "https://www.usatoday.com/story/news/nation-now/2017/08/30/these-most-popular-cars-and-trucks-every-state/478537001/"

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text,'html.parser')

In [3]:
# Retrieve state and model data from "p-text" class
results = soup.find_all('p', class_="p-text")

# Inspect the seventh element 
results[6].text

'Alaska:\xa0F150'

In [4]:
# Create empty list to hold state and model data 
states_cars = [] 

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Extract text
        state_car = result.text
        # Append results to empty list
        states_cars.append(state_car)
        # Preview results
        print(state_car)
    except AttributeError as e:
        print(e)

Ford's F150 is America's best selling vehicle, but the pick-up truck doesn't reign supreme in all 50 states.
According to Kelley Blue Book data, more than 10 states choose SUVs over pickups. Compact cars (Honda's Civic, Toyota's Corolla and Yaris) are the top picks in California, Ohio, Florida, Washington, D.C. and Puerto Rico. 
Still, Kelley Blue Book's list of the most popular vehicles sold in each state based on new car registrations in 2016 shows Americans love their trucks. 
This year, the F-Series remains most popular, but Tim Fleming, analyst for Kelley Blue Book said 2017 consumers could shift last year's data. 
"Sedans are rapidly falling out of favor," Fleming said in an email. "The Camry, Civic and Corolla have dropped from the fourth, fifth and sixth places to seventh, eighth and ninth this year. We don’t see this trend stopping any time soon, although the new Camry could boost sales in the short-term."
Here's a look at the 2016 data:
Alaska: F150
Alabama: F150
Arizona: F15

In [5]:
# Create new list that cotains only state and model information 
states_cars = states_cars[6:58]

### Create DataFrame Using Pandas 

In [6]:
# Create dataframe
df = pd.DataFrame({
    "States_and_Cars":states_cars
})

# Preview dataframe
df.head()

,States_and_Cars
0,Alaska: F150
1,Alabama: F150
2,Arizona: F150
3,Arkansas: Sierra 1500
4,California: Civic


In [7]:
# Separate data into two columns
df=df['States_and_Cars'].str.split(":", n=2, expand = True)

# Rename columns appropriately 
df.rename(columns = {0:'State',1:'Model'},inplace=True)

# Preview dataframe
df.head()

,State,Model
0,Alaska,F150
1,Alabama,F150
2,Arizona,F150
3,Arkansas,Sierra 1500
4,California,Civic


In [8]:
# Create empty list to hold updated car data 
car_list = []

# Loop through and clean from model data
for i in range(len(df['Model'])):
    car = df['Model'][i].replace(u'\xa0',u' ').lstrip().rstrip().replace(' ', '-').replace('F150', 'F-150')
    # Append results to empty list
    car_list.append(car)
# Retrieve unique values from list
set(car_list)

{'Accord',
 'CR-V',
 'Civic',
 'Corolla',
 'Escape',
 'F-150',
 'Outback',
 'RAV4',
 'Rogue',
 'Sierra-1500',
 'Silverado-1500',
 'Tacoma',
 'Yaris'}

In [ ]:
# Add updated model data to dataframe
df['Model'] = car_list

# Preview dataframe
df.head()

,State,Model
0,Alaska,F-150
1,Alabama,F-150
2,Arizona,F-150
3,Arkansas,Sierra-1500
4,California,Civic


In [ ]:
# Add "Make" column that contains the make of each vehicle to dataframe 
df['Make'] = df['Model'].map({
    'Accord' : 'Honda',
    'CR-V' : 'Honda',
    'Civic' : 'Honda',
    'Corolla' : 'Toyota',
    'Escape' : 'Ford',
    'F-150' : 'Ford',
    'Outback' : 'Subaru',
    'RAV4' : 'Toyota',
    'Rogue' : 'Nissan',
    'Sierra-1500' : 'GMC',
    'Silverado-1500' : 'Chevrolet',
    'Tacoma' : 'Toyota',
    'Yaris' : 'Toyota'})

# Preview dataframe
df.head()

,State,Model,Make
0,Alaska,F-150,Ford
1,Alabama,F-150,Ford
2,Arizona,F-150,Ford
3,Arkansas,Sierra-1500,GMC
4,California,Civic,Honda


In [ ]:
# Create dataframe to only contain "Model" and "Make" Columns
unique_cars_df = df[['Model','Make']]

# Drop duplicates
unique_cars_df = unique_cars_df.drop_duplicates(subset='Model')

# Reset index
unique_cars_df.reset_index(drop=True, inplace=True)

# Preview dataframe
unique_cars_df

,Model,Make
0,F-150,Ford
1,Sierra-1500,GMC
2,Civic,Honda
3,Outback,Subaru
4,Rogue,Nissan
5,Silverado-1500,Chevrolet
6,Corolla,Toyota
7,Tacoma,Toyota
8,CR-V,Honda
9,RAV4,Toyota


### Scrape "Motor Trend" Site for Vehicle Information

In [ ]:
# Create two empty lists to hold scraping response and data
car_response = []
car_soup = []

# https://www.motortrend.com/cars/chevrolet/silverado-1500/2019/

# Retrieve Motor Trend page and loop through to receive vehicle information urls for each car
for item in range(len(unique_cars_df['Model'])):
    car_url = "https://www.motortrend.com/cars/" + unique_cars_df['Make'][item] + "/" + unique_cars_df['Model'][item] + \
    "/2016/"
    # Preview url
    print(car_url)
    responses = requests.get(car_url)
    # Append results to both lists
    car_soup.append(bs(responses.text,'html.parser'))
    car_response.append(responses)
    # Preview response
    print(responses)


https://www.motortrend.com/cars/Ford/F-150/2016/
<Response [200]>
https://www.motortrend.com/cars/GMC/Sierra-1500/2016/
<Response [200]>
https://www.motortrend.com/cars/Honda/Civic/2016/
<Response [200]>
https://www.motortrend.com/cars/Subaru/Outback/2016/
<Response [200]>
https://www.motortrend.com/cars/Nissan/Rogue/2016/
<Response [200]>
https://www.motortrend.com/cars/Chevrolet/Silverado-1500/2016/
<Response [200]>
https://www.motortrend.com/cars/Toyota/Corolla/2016/
<Response [200]>
https://www.motortrend.com/cars/Toyota/Tacoma/2016/
<Response [200]>
https://www.motortrend.com/cars/Honda/CR-V/2016/
<Response [200]>
https://www.motortrend.com/cars/Toyota/RAV4/2016/
<Response [200]>
https://www.motortrend.com/cars/Ford/Escape/2016/
<Response [200]>
https://www.motortrend.com/cars/Honda/Accord/2016/
<Response [200]>
https://www.motortrend.com/cars/Toyota/Yaris/2016/
<Response [200]>


In [ ]:
# Retrieve specification headers from "key" tag for first element
car_soup[0].find_all('div', class_ = "key")[1:12]

[<div class="key">Value Rating</div>,
 <div class="key">Engine Name</div>,
 <div class="key">Transmission Name</div>,
 <div class="key">Trim</div>,
 <div class="key">Class</div>,
 <div class="key">Horsepower</div>,
 <div class="key">Standard MPG</div>,
 <div class="key">Body Style</div>,
 <div class="key">Drivetrain</div>,
 <div class="key">Fuel Type</div>,
 <div class="key">Seating Capacity (Std/Max)</div>]

In [ ]:
# Retrieve vehicle specifications from "value" tag for first element
car_soup[0].find_all('div', class_ = "value")[1:12]

[<div class="value">Below Average</div>,
 <div class="value">Engine: V6 Flex Fuel 3.5 Liter</div>,
 <div class="value"><span itemprop="vehicleTransmission">SelectShift® 6 Speed Automatic Mode Select with Manual Mode (6R80E)</span></div>,
 <div class="value"><span itemprop="vehicleConfiguration">XL 2WD Short Bed Regular Cab</span></div>,
 <div class="value">Full-Size Pickup LD</div>,
 <div class="value">283@6500</div>,
 <div class="value"><div itemprop="fuelEfficiency" itemscope="" itemtype="//schema.org/QuantitativeValue">
 <span itemprop="minValue">18</span> City /
 									  <span itemprop="maxValue">24</span> Hwy</div></div>,
 <div class="value"><span itemprop="bodyType">Truck</span></div>,
 <div class="value"><span itemprop="driveWheelConfiguration">RWD</span></div>,
 <div class="value"><span itemprop="fuelType">Unleaded Regular</span></div>,
 <div class="value"><span itemprop="vehicleSeatingCapacity">3/3</span></div>]

In [ ]:
# Retrieve pricing from "price" tag for first element
car_soup[0]('div', class_="price")[2].text

'$21,610'

In [ ]:
# Create empty lists to hold vehicle pricing and specifications 
MSRP = []
CPO = []
UCV = []
VR = []
EN = []
TN = []
T = []
C = []
HP = []
MPG = []
BSt = []
DT = []
FT = []
SC = []
IMGURL = []

# Loop through urls
for item in car_soup:
    # Retrieve pricing from "price" tags
    msr = item.find_all('div', class_="price")[0].text
    try:
        cp = item.find_all('div', class_="price")[1].text        
    except:
        cp = "N/A"
    try:
        uc = item.find_all('div', class_="price")[2].text        
    except:
        uc = "N/A"
        
    # Append pricing to empty lists
    MSRP.append(msr)
    CPO.append(cp)
    UCV.append(uc)
    
    # Retreive vehicle specifications from key value pair tags
    aV = item.find_all('div', class_ = "value")[1].text
    aE = item.find_all('div', class_ = "value")[2].text
    aTN = item.find_all('div', class_ = "value")[3].text
    aT = item.find_all('div', class_ = "value")[4].text
    aC = item.find_all('div', class_ = "value")[5].text
    aHP = item.find_all('div', class_ = "value")[6].text
    aMPG = item.find_all('div', class_ = "value")[7].text
    aBSt = item.find_all('div', class_ = "value")[8].text
    aDT = item.find_all('div', class_ = "value")[9].text
    aFT = item.find_all('div', class_ = "value")[10].text
    aSC = item.find_all('div', class_ = "value")[11].text
    IU = item.find('img', class_ = "attachment-full-width")['data-base']
    
    # Append vehicle specifications to empty lists 
    VR.append(aV)
    EN.append(aE)
    TN.append(aTN)
    T.append(aT)
    C.append(aC)
    HP.append(aHP)
    MPG.append(aMPG.replace('\n', '').replace('\t\t\t\t\t\t\t\t\t ', ''))
    BSt.append(aBSt)
    DT.append(aDT)
    FT.append(aFT)
    SC.append(aSC)
    IMGURL.append(IU)
    
# Preview results
print(MSRP[0])
print(CPO[0])
print(UCV[0])
print(VR[0])
print(EN[0])
print(TN[0])
print(T[0])
print(C[0])
print(HP[0])
print(MPG[0])
print(BSt[0])
print(DT[0])
print(FT[0])
print(SC[0])
print(IMGURL)

$26,540
$23,900
$21,610
Below Average
Engine: V6 Flex Fuel 3.5 Liter
SelectShift® 6 Speed Automatic Mode Select with Manual Mode (6R80E)
XL 2WD Short Bed Regular Cab
Full-Size Pickup LD
283@6500
18 City / 24 Hwy
Truck
RWD
Unleaded Regular
3/3
['https://st.motortrend.com/uploads/sites/10/2017/11/2018-ford-f-150-xlt-supercab-pick-up-angular-front.png', 'https://st.motortrend.com/uploads/sites/10/2018/03/2019-GMC-Sierra-1500-Denali-front-view.jpg', 'https://st.motortrend.com/uploads/sites/10/2018/08/2019-Honda-Civic-Coupe-Sport.jpg', 'https://st.motortrend.com/uploads/sites/10/2017/10/2018-subaru-outback-premium-wagon-angular-front.png', 'https://st.motortrend.com/uploads/sites/10/2017/12/2018-nissan-rogue-s-suv-angular-front.png', 'https://st.motortrend.com/uploads/sites/10/2018/01/2019-Chevrolet-Silverado-1500-front-side-view-1.jpg', 'https://st.motortrend.com/uploads/sites/10/2017/10/2018-toyota-corolla-l-sedan-angular-front.png', 'https://st.motortrend.com/uploads/sites/10/2019/02/202

In [ ]:
# Scrape images
# Create two empty lists to hold scraping response and data
car_response_img = []
car_soup_img = []

# https://www.kbb.com/honda/civic/2016/

# Retrieve Motor Trend page and loop through to receive vehicle information urls for each car
for item in range(len(unique_cars_df['Model'])):
    if unique_cars_df['Model'][item] == 'F-150':
        car_url_img = 'https://www.kbb.com/ford/f150-supercrew-cab/2016/'
    elif unique_cars_df['Model'][item] == 'Sierra-1500':
        car_url_img = 'https://www.kbb.com/gmc/sierra-1500-crew-cab/2016/'
    elif unique_cars_df['Model'][item] == 'Silverado-1500':
        car_url_img = 'https://www.kbb.com/chevrolet/silverado-1500-crew-cab/2016/'
    elif unique_cars_df['Model'][item] == 'Tacoma':
        car_url_img = 'https://www.kbb.com/toyota/tacoma-double-cab/2016/'        
    else:
        car_url_img = "https://www.kbb.com/" + unique_cars_df['Make'][item].lower() + "/" + \
        unique_cars_df['Model'][item].lower() + "/2016/"
    # Preview url
    print(car_url_img)
    responses = requests.get(car_url_img)
    # Append results to both lists
    car_soup_img.append(bs(responses.text,'html.parser'))
    car_response_img.append(responses)
    # Preview response
    print(responses)

https://www.kbb.com/ford/f150-supercrew-cab/2016/
<Response [200]>
https://www.kbb.com/gmc/sierra-1500-crew-cab/2016/
<Response [200]>
https://www.kbb.com/honda/civic/2016/
<Response [200]>
https://www.kbb.com/subaru/outback/2016/
<Response [200]>
https://www.kbb.com/nissan/rogue/2016/


In [ ]:
# Create empty lists to hold vehicle pricing and specifications 
IMGURL = []

# Loop through urls
for item in car_soup_img:
    IU = f"https:{item.find('img', class_ = 'media-hero-image')['src']}"
    
    # Append vehicle image to empty lists 
    IMGURL.append(IU)
    
# Preview results
print(IMGURL[0])

### Create Final DataFrame

In [ ]:
# Create dataframe
df_prelim = pd.DataFrame({
    'Model': unique_cars_df['Model'],
    'Manufacturer_Suggested_Retail_Price':MSRP,
    'Certified_Pre-Owned_Price':CPO,
    'Used_Car_Value':UCV,
    'Value_Rating':VR,
    'Engine_Name':EN,
    'Transmission_Name':TN,
    'Trim':T,
    'Class':C,
    'Horsepower':HP,
    'Standard_MPG':MPG,
    'Body_Style':BSt,
    'Drivetrain':DT,
    'Fuel_Type':FT,
    'Seating_Capacity':SC,
    'Image_URL':IMGURL
})

# Preview dataframe
df_prelim.head()

In [ ]:
# Find cars with missing used price info
# df_prelim[df_prelim.Used_Car_Value == 'N/A']

# Update used price (retrieved manually from kbb)
df_prelim.loc[df_prelim.Model == 'Yaris', 'Used_Car_Value'] = "$9,714"
df_prelim.loc[df_prelim.Model == 'Corolla', 'Used_Car_Value'] = "$13,459"
df_prelim.loc[df_prelim.Model == 'Civic', 'Used_Car_Value'] = "$13,802"
df_prelim.loc[df_prelim.Model == 'Accord', 'Used_Car_Value'] = "$14,222"
# Fix Civic image
df_prelim.loc[df_prelim.Model == 'Civic', 'Image_URL'] = "https://st.motortrend.com/uploads/sites/10/2016/10/2017-Honda-Civic-sedan-front-three-quarter.jpg"

# preview dataframe
df_prelim.head(50)

In [ ]:
df.head()

In [ ]:
# Conduct a "left" join to merge dataframes
final_df = df.merge(df_prelim, how = 'left', on = 'Model')

# Preview final dataframe
final_df.head()

### Update Map Information JSON File

In [ ]:
# Open JSON as a read only and load into variable
with open('state_boundaries.json', 'r') as f:
    datastore = json.load(f)

In [ ]:
# Extract desired columns from dataframe
panel_df = final_df[['State', 'Model', 'Make', 'Manufacturer_Suggested_Retail_Price', 'Class', 'Standard_MPG', \
                      'Body_Style', 'Image_URL']]

# Rename columns appropriately 
panel_df.columns = ['STATE', 'MODEL', 'MAKE', 'MSRP', 'CLASS', 'MPG', 'BODY', 'IMGURL']

# Convert dataframe to dictionary format
car_info_dict = panel_df.to_dict(orient='records')

# Inspect dictionary
car_info_dict

In [ ]:
# Loop through "features" dictionary
for feature in datastore['features']:
# Print(feature['properties']['NAME'])
    for entry in car_info_dict:
        if entry['STATE'] == feature['properties']['NAME']:
            feature['properties'].update(entry)
# Inspect first element    
datastore['features'][0]

In [ ]:
# Create new JSON file with new information
with open('state_car_info.json', 'w') as fp:
    json.dump(datastore, fp)

### Create Database

In [ ]:
# Create SQLite DB
sql_data = 'db\cars.sqlite' 

# Connect to DB
conn = sq.connect(sql_data)
cur = conn.cursor()

# Drop table if it exists and create it from dataframe
cur.execute('''DROP TABLE IF EXISTS car_stats''')
final_df.to_sql('car_stats', conn, if_exists='replace', index=False) # - writes the pd.df to SQLIte DB
pd.read_sql('select * from car_stats', conn)

# Commit and close connection
conn.commit()
conn.close()

In [ ]:
# Query DB table
conn = sq.connect(sql_data)
pd.read_sql('select * from car_stats', conn)

In [ ]:
# plot_results = db.session.query(Cars.Make, Cars.Model, Cars.Manufacturer_Suggested_Retail_Price, \
#                                 Cars.Used_Car_Value).distinct()

# car_info = []
# plot_info = []

# for result in plot_results:
#     car_info.append(f'{result[0]} {result[1]}')
#     car_info.append(result[2].replace('$','').replace(',',''))
#     car_info.append(result[3].replace('$','').replace(',',''))
#     plot_info.append(car_info)
#     car_info = []
    
# # Distinct list of lists    
# import itertools
# plot_info.sort()
# list(plot_info for plot_info,_ in itertools.groupby(plot_info))    